In [68]:
import os
import random
from typing import List
import twscrape

In [23]:
# set up directories, e.g. for 2008

year = 2008
os.mkdir(str(year))

# single digit month
for i in range(1,10):
    path = str(year) + "/" + str(year) + "-0" + str(i)
    os.mkdir(path) 

# double digit month
for j in range(10,13):
    path = str(year) + "/" + str(year) + "-" + str(j)
    os.mkdir(path) 

In [22]:
# remove directories if needed
!rm -rf 2008/

In [70]:
# collect tweets for 10 randomly sampled days of month at randomly sampled time of day

def scrape_tweets(year: str, months: List[str], days_in_month: int):
    range_days = list(range(1,days_in_month+1))
    range_times = list(range(0,24))

    # sample 10 random days and times of day for each month
    for m in months: 
        month_path = year + "/" + year + "-" + m
        days = sorted(random.sample(range_days, k=10))   # random days of month (no replacement)
        times = random.choices(range_times, k=10)   # random hours of day (replacement)
        
        # scrape tweets for the 10 picked days and times
        for t in range(10):     
            day = "0" + str(days[t]) if days[t] < 10 else str(days[t])
            time = "0" + str(times[t]) if times[t] < 10 else str(times[t])
            day_path = month_path + "/" + year + "-" + m + "-" + day + ".txt"
            command = 'twscrape search "since:' + year + '-' + m + '-' + day + '_' + time + ':00:00_UTC lang:ja"　> ' + day_path + ' --limit=4500'
            # os.system(command)

# 28 day months
scrape_tweets("2015",["02"], 28)    # omit leap year 29th days for simplicity...?

# 30 day months
months_30 = ["04","06","09","11"]
scrape_tweets("2015",months_30, 30)

# 31 day months
months_31 = ["01","03","05","07","08","10","12"]
scrape_tweets("2015",months_31, 31)

twscrape search "since:2015-02-02_12:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-02.txt --limit=4500
twscrape search "since:2015-02-04_20:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-04.txt --limit=4500
twscrape search "since:2015-02-07_02:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-07.txt --limit=4500
twscrape search "since:2015-02-10_00:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-10.txt --limit=4500
twscrape search "since:2015-02-14_13:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-14.txt --limit=4500
twscrape search "since:2015-02-16_12:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-16.txt --limit=4500
twscrape search "since:2015-02-19_18:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-19.txt --limit=4500
twscrape search "since:2015-02-21_06:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-21.txt --limit=4500
twscrape search "since:2015-02-22_08:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-22.txt --limit=4500
twscrape search "since:2015-02-28_08:00:00_UTC lang:ja"　> 2015/2015-02/2015-02-28.txt --limit=4500
twscrape s